### Restart and run all cells

### stock: dividend, price

In [1]:
import calendar
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 2, 8)

In [2]:
format_dict = {
               'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
               'price_x':'{:,.2f}','to_price':'{:,.2f}',
               'current':'{:,.2f}','div_amt':'{:,.2f}','amount':'{:,.2f}','yield':'{:,.2f}',
               'Q4':'{:,.2f}','Q3':'{:,.2f}','Q2':'{:,.2f}','Q1':'{:,.2f}','DIVIDEND':'{:,.2f}',
               'qty':'{:,}','shares':'{:,}','qty_z':'{:,}'
              }
pd.set_option("display.max_rows", None)

In [3]:
sql = '''
SELECT *
FROM dividend
ORDER BY name'''
df_dividend = pd.read_sql(sql, const)
df_dividend.shape

(78, 13)

In [4]:
### To find dividend without price record
sql = '''
SELECT D.name, P.name
FROM dividend D
LEFT JOIN price P
ON D.name = P.name
AND P.date = "%s"
WHERE P.name IS NULL
'''
sql = sql % today
print(sql)

df_left = pd.read_sql(sql, const)
df_left


SELECT D.name, P.name
FROM dividend D
LEFT JOIN price P
ON D.name = P.name
AND P.date = "2022-02-08"
WHERE P.name IS NULL



,name,name


### If above statement consists stock names, must delete from dividend as there are no price records

In [5]:
sql = '''
SELECT D.name, FORMAT(DIVIDEND/P.price*100,2) AS yield,
Q4, Q3, Q2, Q1, DIVIDEND, shares, DIVIDEND * shares AS div_amt,
P.price AS current, shares*P.price AS amount, XDATE, PAIDDATE, ACTUAL
FROM dividend D
JOIN price P
WHERE D.name = P.name
AND P.date = "%s"'''
sql = sql % today
print(sql)

div_price = pd.read_sql(sql, const)
div_price['yield'] = div_price['yield'].astype(float)
div_price['shares'] = div_price['shares'].astype(int)
div_price.shape


SELECT D.name, FORMAT(DIVIDEND/P.price*100,2) AS yield,
Q4, Q3, Q2, Q1, DIVIDEND, shares, DIVIDEND * shares AS div_amt,
P.price AS current, shares*P.price AS amount, XDATE, PAIDDATE, ACTUAL
FROM dividend D
JOIN price P
WHERE D.name = P.name
AND P.date = "2022-02-08"


(78, 14)

In [6]:
div_price.nlargest(3, 'div_amt')[['name','div_amt']]

,name,div_amt
35,STA,105000.0
48,JASIF,95000.0
2,MCS,68500.0


In [7]:
y_ovr_X = div_price['yield'] >= 7.00
div_price[y_ovr_X].sort_values(['yield'],ascending=[False]).style.format(format_dict)

,name,yield,Q4,Q3,Q2,Q1,DIVIDEND,shares,div_amt,current,amount,XDATE,PAIDDATE,ACTUAL
35,STA,16.54,1.75,1.25,1.25,1.00,5.25,"20,000","105,000.00",31.75,"635,000.00",2021-12-08,2021-12-08,0
33,STGT,15.00,2.00,1.25,1.25,0.00,4.50,0,0.00,30.00,0.00,2021-12-07,2021-12-07,0
46,NOBLE,13.49,0.50,0.00,0.35,0.00,0.85,"63,000","53,550.00",6.30,"396,900.00",2022-05-10,2021-09-07,0
24,TMT,10.68,0.50,0.00,0.60,0.00,1.10,"27,000","29,700.00",10.30,"278,100.00",2021-09-10,2021-09-10,0
2,MCS,9.32,0.60,0.37,0.40,0.00,1.37,"50,000","68,500.00",14.70,"735,000.00",2022-03-15,2021-12-09,0
48,JASIF,8.80,0.25,0.22,0.24,0.24,0.95,"100,000","95,000.00",10.80,"1,080,000.00",2022-02-24,2021-12-08,0
61,WIIK,8.26,0.09,0.00,0.10,0.00,0.19,0,0.00,2.30,0.00,2021-05-21,2021-05-21,0
77,RCL,8.05,0.50,1.50,1.50,0.00,3.50,"4,500","15,750.00",43.50,"195,750.00",2021-12-03,2021-12-03,0
68,HREIT,7.97,0.17,0.17,0.17,0.17,0.69,"30,000","20,796.00",8.70,"261,000.00",2022-03-01,2021-12-23,0
6,PDG,7.96,0.22,0.00,0.10,0.00,0.32,0,0.00,4.02,0.00,2022-03-15,2021-05-20,0


In [8]:
not_due = div_price['ACTUAL'] == 1
div_price[not_due].sort_values(['yield'],ascending=[False]).style.format(format_dict)

,name,yield,Q4,Q3,Q2,Q1,DIVIDEND,shares,div_amt,current,amount,XDATE,PAIDDATE,ACTUAL
67,CPNCG,7.55,0.22,0.24,0.24,0.25,0.96,"10,000","9,587.00",12.70,"127,000.00",2022-02-14,2022-03-04,1
30,DIF,7.20,0.26,0.26,0.26,0.26,1.04,"60,000","62,640.00",14.50,"870,000.00",2022-02-10,2022-03-04,1
9,SCC,4.78,10.00,0.00,8.50,0.00,18.50,900,"16,650.00",387.00,"348,300.00",2022-04-07,2022-04-26,1
7,DTAC,4.49,1.05,0.00,1.05,0.00,2.10,0,0.00,46.75,0.00,2022-02-11,2022-04-22,1
74,PTTEP,3.73,3.00,0.00,2.00,0.00,5.00,0,0.00,134.00,0.00,2022-02-10,2022-04-18,1
71,SCB,3.12,2.63,0.00,1.43,0.00,4.06,0,0.00,130.00,0.00,2022-02-14,2022-04-22,1


In [9]:
div_price[y_ovr_X & not_due].sort_values(['XDATE','name'],ascending=[True,True]).style.format(format_dict)

,name,yield,Q4,Q3,Q2,Q1,DIVIDEND,shares,div_amt,current,amount,XDATE,PAIDDATE,ACTUAL
30,DIF,7.20,0.26,0.26,0.26,0.26,1.04,"60,000","62,640.00",14.50,"870,000.00",2022-02-10,2022-03-04,1
67,CPNCG,7.55,0.22,0.24,0.24,0.25,0.96,"10,000","9,587.00",12.70,"127,000.00",2022-02-14,2022-03-04,1


In [10]:
in_port = div_price['shares'] > 0
div_price[in_port].sort_values(['name'],ascending=[True]).style.format(format_dict)

,name,yield,Q4,Q3,Q2,Q1,DIVIDEND,shares,div_amt,current,amount,XDATE,PAIDDATE,ACTUAL
54,BCH,1.79,0.13,0.00,0.20,0.00,0.33,"15,000","4,950.00",18.40,"276,000.00",2021-09-15,2021-09-15,0
73,BGRIM,1.34,0.30,0.00,0.15,0.00,0.45,"9,000","4,050.00",33.50,"301,500.00",2021-05-11,2021-05-11,0
72,CKP,0.67,0.04,0.00,0.00,0.00,0.04,"20,000",700.00,5.20,"104,000.00",2022-05-20,2022-05-20,0
67,CPNCG,7.55,0.22,0.24,0.24,0.25,0.96,"10,000","9,587.00",12.70,"127,000.00",2022-02-14,2022-03-04,1
30,DIF,7.20,0.26,0.26,0.26,0.26,1.04,"60,000","62,640.00",14.50,"870,000.00",2022-02-10,2022-03-04,1
23,DOHOME,0.45,0.10,0.00,0.00,0.00,0.10,"12,000","1,200.00",22.10,"265,200.00",2021-05-21,2021-05-21,0
58,EPG,2.15,0.00,0.14,0.00,0.09,0.23,"24,000","5,520.00",10.70,"256,800.00",2021-12-08,2021-12-08,0
68,HREIT,7.97,0.17,0.17,0.17,0.17,0.69,"30,000","20,796.00",8.70,"261,000.00",2022-03-01,2021-12-23,0
10,IP,0.84,0.18,0.00,0.00,0.00,0.18,"7,500","1,387.50",22.00,"165,000.00",2022-04-27,2022-04-27,0
13,IVL,1.72,0.25,0.25,0.17,0.17,0.85,"6,000","5,100.00",49.50,"297,000.00",2022-05-06,2021-12-09,0


In [11]:
div_price[y_ovr_X & in_port].sort_values(['yield','name'],ascending=[False,True]).style.format(format_dict)

,name,yield,Q4,Q3,Q2,Q1,DIVIDEND,shares,div_amt,current,amount,XDATE,PAIDDATE,ACTUAL
35,STA,16.54,1.75,1.25,1.25,1.00,5.25,"20,000","105,000.00",31.75,"635,000.00",2021-12-08,2021-12-08,0
46,NOBLE,13.49,0.50,0.00,0.35,0.00,0.85,"63,000","53,550.00",6.30,"396,900.00",2022-05-10,2021-09-07,0
24,TMT,10.68,0.50,0.00,0.60,0.00,1.10,"27,000","29,700.00",10.30,"278,100.00",2021-09-10,2021-09-10,0
2,MCS,9.32,0.60,0.37,0.40,0.00,1.37,"50,000","68,500.00",14.70,"735,000.00",2022-03-15,2021-12-09,0
48,JASIF,8.80,0.25,0.22,0.24,0.24,0.95,"100,000","95,000.00",10.80,"1,080,000.00",2022-02-24,2021-12-08,0
77,RCL,8.05,0.50,1.50,1.50,0.00,3.50,"4,500","15,750.00",43.50,"195,750.00",2021-12-03,2021-12-03,0
68,HREIT,7.97,0.17,0.17,0.17,0.17,0.69,"30,000","20,796.00",8.70,"261,000.00",2022-03-01,2021-12-23,0
67,CPNCG,7.55,0.22,0.24,0.24,0.25,0.96,"10,000","9,587.00",12.70,"127,000.00",2022-02-14,2022-03-04,1
30,DIF,7.20,0.26,0.26,0.26,0.26,1.04,"60,000","62,640.00",14.50,"870,000.00",2022-02-10,2022-03-04,1
19,RATCH,7.18,1.25,0.00,2.00,0.00,3.25,"9,000","29,250.00",45.25,"407,250.00",2022-03-02,2021-09-23,0


In [12]:
dividend = div_price[y_ovr_X & in_port].div_amt.sum()
cost = div_price[y_ovr_X & in_port].amount.sum()
yld = round(dividend / cost * 100,2)
(dividend, cost, yld)

(498071.0, 5103000.0, 9.76)

In [13]:
file_name = 'DIV-OVR-X%.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

div_price[y_ovr_X & in_port].sort_values(['yield','name'],ascending=[False,True]).to_csv(data_file, index=False)
div_price[y_ovr_X & in_port].sort_values(['yield','name'],ascending=[False,True]).to_csv(output_file, index=False)
div_price[y_ovr_X & in_port].sort_values(['yield','name'],ascending=[False,True]).to_csv(box_file, index=False)
div_price[y_ovr_X & in_port].sort_values(['yield','name'],ascending=[False,True]).to_csv(one_file, index=False)

In [14]:
div_price[~y_ovr_X & in_port].sort_values(['yield','name'],ascending=[False,True]).style.format(format_dict)

,name,yield,Q4,Q3,Q2,Q1,DIVIDEND,shares,div_amt,current,amount,XDATE,PAIDDATE,ACTUAL
69,TISCO,6.41,6.30,0.00,0.00,0.00,6.30,"2,000","12,600.00",98.25,"196,500.00",2022-04-28,2021-05-17,0
75,MC,5.91,0.00,0.20,0.00,0.35,0.55,"10,000","5,500.00",9.30,"93,000.00",2022-02-25,2021-11-25,0
62,LPF,5.74,0.17,0.19,0.18,0.19,0.73,"10,000","7,284.00",12.70,"127,000.00",2022-01-28,2022-02-15,0
76,RJH,5.27,0.20,1.10,0.40,0.00,1.70,"4,500","7,650.00",32.25,"145,125.00",2021-12-07,2021-12-07,0
51,PTTGC,5.22,1.00,0.00,2.00,0.00,3.00,"6,000","18,000.00",57.50,"345,000.00",2022-02-25,2021-09-22,0
16,PTT,5.02,0.82,0.00,1.20,0.00,2.02,"9,000","18,180.00",40.25,"362,250.00",2021-10-15,2021-10-15,0
9,SCC,4.78,10.00,0.00,8.50,0.00,18.50,900,"16,650.00",387.00,"348,300.00",2022-04-07,2022-04-26,1
55,TU,3.97,0.40,0.00,0.45,0.00,0.85,"18,000","15,300.00",21.40,"385,200.00",2022-03-08,2021-09-07,0
14,NER,3.03,0.15,0.00,0.07,0.00,0.22,"27,000","5,940.00",7.25,"195,750.00",2021-09-06,2021-09-06,0
47,SIS,2.52,1.00,0.00,0.00,0.00,1.00,"4,500","4,500.00",39.75,"178,875.00",2021-05-14,2021-05-14,0


In [15]:
dividend = div_price[~y_ovr_X & in_port].div_amt.sum()
cost = div_price[~y_ovr_X & in_port].amount.sum()
yld = round(dividend / cost * 100,2)
(dividend, cost, yld)

(150931.5, 4953500.0, 3.05)